У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [53]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('data/customer_segmentation_train.csv')

In [3]:
df.head(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [5]:
df.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

Створю базову функцію для виводу основної інформації по стовпцю.

In [6]:
# Функція для аналізу стовпця
def column_analysis(column_name):
    percent_of_missing_value = (df[column_name].isnull().sum() / len(df) * 100).round(2)
    percent_of_each_value_in_column = df[column_name].value_counts(normalize=True).round(2)
    print('Відсоток пропущених значень: ', percent_of_missing_value)
    print('Відсоток наявних значень у колонці: ',percent_of_each_value_in_column)

Аналіз стовпця `Ever_Married`

In [7]:
column_analysis('Ever_Married')

Відсоток пропущених значень:  1.74
Відсоток наявних значень у колонці:  Ever_Married
Yes    0.59
No     0.41
Name: proportion, dtype: float64


Вирішую заповнити пропущені дані випадковим чином у співвідношені 59/41 - Yes/No відповідно і одразу трансформувати це в 1/0.

In [8]:
# Рахую кількість пропущених значень у стовпчику 'Ever_Married'
num_missing = df['Ever_Married'].isnull().sum()

# Генерую випадковим чином дані 
random_values = np.random.choice(['Yes', 'No'], size=num_missing, p=[0.59, 0.41])

# Заповнюю пропущені значення цими випадковими величинами
df.loc[df['Ever_Married'].isnull(), 'Ever_Married'] = random_values

# Трансформую "Так" на 1 і "Ні" на 0
df['Ever_Married'] = df['Ever_Married'].map({'Yes': 1, 'No': 0})

Вирішую одразу закодувати `Gender`. Для цього перевірю якого класу більше.

In [9]:
df.Gender.value_counts()

Gender
Male      4417
Female    3651
Name: count, dtype: int64

In [10]:
# Трансформую "Male" на 1 і "Female" на 0 у новому стовпчику "Is_Male"
df['Is_Male'] = df['Gender'].map({'Male': 1, 'Female': 0})

In [11]:
# Видаляю стовпчик Gender
df.drop(columns=['Gender'], inplace=True)

In [12]:
df.head(5)

,ID,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation,Is_Male
0,462809,0,22,No,Healthcare,1.0,Low,4.0,Cat_4,D,1
1,462643,1,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A,0
2,466315,1,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B,0
3,461735,1,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B,1
4,462669,1,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A,0


`Age` не має відсутніх значень. Залишаю поки без змін.
Перехожу до аналізу колонки `Graduated`

In [13]:
column_analysis('Graduated')

Відсоток пропущених значень:  0.97
Відсоток наявних значень у колонці:  Graduated
Yes    0.62
No     0.38
Name: proportion, dtype: float64


Пропущених значень <1%. 62% заповнених значень - Yes, 38% - No. Заповнюю дані аналогічно колонці `Ever_Married`

In [14]:
num_missing = df['Graduated'].isnull().sum()

# Генерую випадковим чином дані 
random_values = np.random.choice(['Yes', 'No'], size=num_missing, p=[0.62, 0.38])

# Заповнюю пропущені значення цими випадковими величинами
df.loc[df['Graduated'].isnull(), 'Graduated'] = random_values

# Рахую кількість пропущених значень у стовпчику 'Ever_Married'
num_missing = df['Graduated'].isnull().sum()

# Трансформую "Так" на 1 і "Ні" на 0
df['Graduated'] = df['Graduated'].map({'Yes': 1, 'No': 0})

Перехожу до аналізу стовпця `Profession`.

In [15]:
column_analysis('Profession')

Відсоток пропущених значень:  1.54
Відсоток наявних значень у колонці:  Profession
Artist           0.32
Healthcare       0.17
Entertainment    0.12
Engineer         0.09
Doctor           0.09
Lawyer           0.08
Executive        0.08
Marketing        0.04
Homemaker        0.03
Name: proportion, dtype: float64


Оскільки пропущені значення можуть означати юезробітність, і відсоток пропущених даних >1,5%. Приймаю рішення заповнити пропущенні значення як 'Unknown'

In [16]:
# Заповнюю відсутні значення у стовпчику "Profession" значенням "Unknown"
df['Profession'] = df['Profession'].fillna('Unknown')

Аналіз стовпця `Work_Experience`.

In [17]:
column_analysis('Work_Experience')

Відсоток пропущених значень:  10.28
Відсоток наявних значень у колонці:  Work_Experience
1.0     0.33
0.0     0.32
9.0     0.07
8.0     0.06
2.0     0.04
3.0     0.04
4.0     0.03
6.0     0.03
7.0     0.03
5.0     0.03
10.0    0.01
11.0    0.01
12.0    0.01
13.0    0.01
14.0    0.01
Name: proportion, dtype: float64


In [18]:
df['Work_Experience'].describe()

count    7239.000000
mean        2.641663
std         3.406763
min         0.000000
25%         0.000000
50%         1.000000
75%         4.000000
max        14.000000
Name: Work_Experience, dtype: float64

In [19]:
df['Work_Experience'].mode()

0    1.0
Name: Work_Experience, dtype: float64

Мода та медіана - 1. Середнє значення 2,64. Вирішую заповнити пропущенні значення модою. 

In [20]:
# Заповнюю відсутні значення у стовпчику "Work_Experience" значенням "1.0"
df['Work_Experience'] = df['Work_Experience'].fillna(1.0)

Аналіз стовпця `Spending_Score`

In [21]:
column_analysis('Spending_Score')

Відсоток пропущених значень:  0.0
Відсоток наявних значень у колонці:  Spending_Score
Low        0.60
Average    0.24
High       0.15
Name: proportion, dtype: float64


Пропущених значень немає. Перехожу до `Family_Size`.

In [22]:
column_analysis('Family_Size')

Відсоток пропущених значень:  4.15
Відсоток наявних значень у колонці:  Family_Size
2.0    0.31
3.0    0.19
1.0    0.19
4.0    0.18
5.0    0.08
6.0    0.03
7.0    0.01
8.0    0.01
9.0    0.01
Name: proportion, dtype: float64


In [23]:
df['Family_Size'].describe()

count    7733.000000
mean        2.850123
std         1.531413
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         9.000000
Name: Family_Size, dtype: float64

In [24]:
df['Family_Size'].mode()

0    2.0
Name: Family_Size, dtype: float64

Мода - 2. Середнє значення 2,85, медіана - 3. Вирішую заповнити пропущенні значення модою.

In [25]:
# Заповнюю відсутні значення у стовпчику "Family_Size" значенням "2.0"
df['Family_Size'] = df['Family_Size'].fillna(2.0)

Аналіз стовпця `Var_1`.

In [26]:
column_analysis('Var_1')

Відсоток пропущених значень:  0.94
Відсоток наявних значень у колонці:  Var_1
Cat_6    0.66
Cat_4    0.14
Cat_3    0.10
Cat_2    0.05
Cat_7    0.03
Cat_1    0.02
Cat_5    0.01
Name: proportion, dtype: float64


Оскільки відсоток пропущених значень (`0.94%`) незначний. Вирішую заповнити значення модою (`Cat_6`), яка займає `66%` даних.

In [27]:
# Заповнюю відсутні значення у стовпчику "Var_1" значенням "Cat_6"
df['Var_1'] = df['Var_1'].fillna('Cat_6')

Колонка `Segmentation` не має відсутніх значень.

Перехожу до кодування категоріальних колонок.

In [28]:
cat_cols = list(df.select_dtypes(include='object').columns)

In [29]:
cat_cols.remove('Segmentation')

In [30]:
encoder = OneHotEncoder(sparse_output=False)

In [31]:
encoded_cat_cols = encoder.fit(df[cat_cols]).get_feature_names_out()

In [32]:
df[encoded_cat_cols] = encoder.transform(df[cat_cols])

In [33]:
# Видаляю категоріальні колонки та колонку ID
df.drop(columns=(cat_cols), inplace=True)

In [34]:
# Видаляю колонку ID
df.drop(columns=('ID'), inplace=True)

Заскейлю колонки `Age`, `Family_Size` та `Work_Experience`.

In [35]:
num_cols = ['Age', 'Family_Size', 'Work_Experience']
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

Кодую цільову колонку Segmentation.

In [36]:
le = LabelEncoder()
df['Segmentation_encoded'] = le.fit_transform(df['Segmentation'])

In [74]:
label_encoders = {}
label_encoders['Segmentation'] = le

In [37]:
# Видаляю стовпчик Segmentation
df.drop(columns=['Segmentation'], inplace=True)

Розбиваю на тренувальну і тестувальну вибірку.

In [38]:
X_train, X_validation, y_train, y_validation = train_test_split(df.drop(columns=['Segmentation_encoded']), 
                                                                df['Segmentation_encoded'], 
                                                                test_size=0.2, 
                                                                stratify=df['Segmentation_encoded'], 
                                                                random_state=29)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

Створюю тренувальний набір із методом SMOTENC.

In [39]:
categorical_columns = df.drop(columns=num_cols, axis=1).columns.to_list()

In [40]:
# Визначаю індекси категоріальних ознак
categorical_columns_indices = [df.columns.get_loc(col) for col in categorical_columns[:-1]]

In [41]:
# Ініціація SMOTENC
smotenc = SMOTENC(categorical_features=categorical_columns_indices, random_state=0)

In [42]:
smotenc

SMOTENC(categorical_features=[0, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                              17, 18, 19, 20, 21, 22, 23, 24, 25],
        random_state=0)

In [43]:
# Fit та resample тренувальних даних
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

In [48]:
# Перевіряю розподіл цільових даних
print("Початковий розподіл:\n", y_train.value_counts())
print("Розподіл після апсемплингу:\n", pd.Series(y_train_smotenc).value_counts())

Початковий розподіл:
 Segmentation_encoded
3    1814
0    1578
2    1576
1    1486
Name: count, dtype: int64
Розподіл після апсемплингу:
 Segmentation_encoded
2    1814
3    1814
1    1814
0    1814
Name: count, dtype: int64


Створюю тренувальний набір із методом SMOTE-Tomek.

In [46]:
smotetomek = SMOTETomek(random_state=0)

In [47]:
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, y_train)

In [50]:
# Перевіряю розподіл цільових даних
print("Початковий розподіл:\n", y_train.value_counts())
print("Розподіл після ресемплингу:\n", pd.Series(y_train_smotetomek).value_counts())

Початковий розподіл:
 Segmentation_encoded
3    1814
0    1578
2    1576
1    1486
Name: count, dtype: int64
Розподіл після ресемплингу:
 Segmentation_encoded
2    1468
3    1456
0    1402
1    1378
Name: count, dtype: int64


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [52]:
model = LogisticRegression(multi_class='ovr', solver='liblinear')

In [76]:
def predict(X_train, y_train, X_validation, y_validation):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_validation)
    report = classification_report(y_validation, y_pred, target_names=label_encoders['Segmentation'].classes_)
    print(report)

In [77]:
train_data_sets = {
    'original_data': (X_train, y_train, X_validation, y_validation),
    'smotenc_data': (X_train_smotenc, y_train_smotenc, X_validation, y_validation),
    'smotetomek_data': (X_train_smotetomek, y_train_smotetomek, X_validation, y_validation)
}

In [78]:
for key, data_sets in train_data_sets.items():
    print(f"Оцінка {key}")
    predict(*data_sets)

Оцінка original_data
              precision    recall  f1-score   support

           A       0.38      0.48      0.42       394
           B       0.39      0.15      0.22       372
           C       0.51      0.63      0.57       394
           D       0.64      0.68      0.66       454

    accuracy                           0.50      1614
   macro avg       0.48      0.49      0.47      1614
weighted avg       0.49      0.50      0.48      1614

Оцінка smotenc_data
              precision    recall  f1-score   support

           A       0.39      0.48      0.43       394
           B       0.40      0.23      0.29       372
           C       0.53      0.61      0.56       394
           D       0.66      0.65      0.66       454

    accuracy                           0.50      1614
   macro avg       0.49      0.49      0.49      1614
weighted avg       0.50      0.50      0.49      1614

Оцінка smotetomek_data
              precision    recall  f1-score   support

           

**Висновок:** 
- original_data: Клас D має найвищу точність `0,64` та recall `0,68`. Клас B має найнижчий recall `0,15`, що свідчить про складність правильного прогнозування цього класу. Accuracy `0,50`.

- SMOTENC: Незначне покращення recall для класу B `0,23` порівняно з початковими даними. Accuracy залишається незмінною `0,50`, але macro avg та weighted avg для f1-score трохи покращені. 

- SMOTETOMEK: Загалом, показники ефективності подібні до даних SMOTENC.

Я би обрала метрику `F1-Score` для оцінювання результатів. Оскільки вона враховує враховує `precision` та `recall`. 

Суттєвої різниці між моделями немає. Усі три набори даних показують найвищий показник `F1` для класу `D` і найнижчий для класу `B`. Трішки гірші результати показала модель натренована на оригінальних даних. Можлив, ознаки в оригінальному наборі даних не є інформативними для цільової змінної, в такому випадку повторна вибірка не призведе до значного покращення результатів.